# From Maps to Models - Tutorials for structural geological modeling using GemPy and GemGIS

<img src="../data/images/gempy_logo.png" />


Welcome to the course "From Maps to Models - Tutorials for structural geological modeling using GemPy and GemGIS". In this course, you will learn how to obtain and process spatial data with the Python open-source package [GemGIS](https://github.com/cgre-aachen/gemgis), how to create structural geological models using the Python open-source package [GemPy](https://github.com/cgre-aachen/gempy) and how to post process the models to obtain certain information from your models. Both packages were developed and are maintained by departments of Prof. Florian Wellmann at [RWTH Aachen University](https://www.cg3.rwth-aachen.de/cms/~qoyf/cg3/?lidx=1) and [Fraunhofer IEG](https://www.ieg.fraunhofer.de/de/geschaeftsbereiche/georessourcen.html). 

The course is divided into four sections. 
1. [Introduction to structural geological modeling using GemPy and GemGIS](#introduction-to-structural-geological-modeling-using-gempy-and-gemgis)
    1. [Introdurction to structural geological models](#introduction-to-structural-geological-models)
    2. [Introduction to GemPy and GemGIS](#introduction-to-gempy-and-gemgis)
    3. [Resources for GemPy and GemGIS](#resources-for-gemgis-gempy)
    4. [Installation of GemPy and GemGIS](#installation-of-gempy-and-gemgis)
2. Modeling of the basic structures in GemPy
    1. Modeling planar layers
    2. Modeling folded layers
    3. Modeling faulted layers
    4. Modeling truncated/unconformal layers
3. Modeling of more complex models including additional tasks (main part)
4. Post-processing of structural geological models using GemGIS

In this notebook, the Introduction to structural geological modeling using GemPy and GemGIS is covered.

<a id='introduction-to-structural-geological-modeling-using-gempy-and-gemgis'></a>
# Introduction to structural geological modeling using GemPy and GemGIS

The following sections provide an introduction to geological models, the Python open-source packages GemPy und GemGIS, installation instructions to create a running `conda` environment and further resources to get started with GemPy and GemGIS. 

<a id='introduction-to-structural-geological-models'></a>
## Introduction to structural geological models

Structural geological models (sensu [Wellmann & Caumon, 2018](https://doi.org/10.1016/bs.agph.2018.09.001)) represent geometric elements in the subsurface, so-called surfaces (Fig. 1). These surfaces may be lithological boundaries such as a boundary between a sandstone or a claystone, which are used mostly in this course, stratigraphic boundaries between layers of different age, which are mostly used in the real world, faults separating a continuous layer or intrusions. A layer is defined here as rock mass exhibiting the same properties such as lithology or stratigraphic age. A layer is bound by a top and a bottom surface. The structural geological model is created in the first place according to the stratigraphic principles of Steno (1669) that more recent strata are deposited roughly horizontally on top of older strata and that variations such as folded, faulted or unconformable layers are caused by geological event. 

<img src="../data/images/fig1.png" width=500/>

Fig. 1: Geological model of the Perth basin (Australia) rendered using GemPy on the in-built Python in Blender, spheres and cones represent the input data. From [de la Varga et al. (2019)](https://doi.org/10.5194/gmd-12-1-2019).

The course of the layers can be observed on the surface in outcrops or quarries (Fig. 2). Further, direct observations of layer boundaries can be made in the subsurface in boreholes or indirectly using geophysical methods such as seismic representing the seismic impedance (product of rock density and seismic velocity). This course focuses on observations on the surface and in boreholes.


<img src="../data/images/fig2.JPG" width=500/>

Fig. 2: Quarry Binsfeldhammer, Stolberg, Aachen City Region, Germany.

The information that can be obtained from observed layer boundaries are the locations of these boundaries (`x`,`y`,`z`) and which boundary they represent (for this course termed `formation`), and the orientation of this layer at a measured location expressed as `dip` and `azimuth` (Fig. 3). The locations should be expressed in meters (m) rather than geographical coordinates (longitude/latitude). The dip and azimuth are expressed in degrees. The dip varies from 0° for horizontal layers to 90° for vertical layers. The azimuth varies from 0° (N) via 180° (S) to 360° (N). 


<img src="../data/images/fig3.png" width=500/>

Fig. 3: Example of strike and dip on tilted sedimentary beds.

By CrunchyRocks, after Karla Panchuck - https://openpress.usask.ca/physicalgeology/chapter/13-5-measuring-geological-structures/, CC BY 4.0, https://commons.wikimedia.org/w/index.php?curid=113554289


<a id='introduction-to-gempy-and-gemgis'></a>
## Introduction to GemPy and GemGIS

GemPy and GemGIS are two Python open-source packages developed and are maintained by departments of Prof. Florian Wellmann at [RWTH Aachen University](https://www.cg3.rwth-aachen.de/cms/~qoyf/cg3/?lidx=1) and [Fraunhofer IEG](https://www.ieg.fraunhofer.de/de/geschaeftsbereiche/georessourcen.html). GemPy is capable of creating structural geological models based on an implicit modeling approach using universal co-kriging [(de la Varga et al., 2019)](https://doi.org/10.5194/gmd-12-1-2019) and the potential field approach by [Lajaunie et al. (1997)](https://doi.org/10.1007/BF02775087). 

A simple synclinal model is shown to illustrate the principles of GemPy (Fig. 4, left). The first input data used for the interpolation of geological surfaces, here stratigraphic surfaces, are the location (x, y, z) of stratigraphic boundaries mapped in surface outcrops, interpolated on geological maps, in wells or seismic data in the subsurface, or constrained from additional geophysical methods (e.g. gravity, ERT). Orientation data (x, y, z, dip, azimuth) obtained from outcrop measurements or reference values from literature are the second type of input data (Fig. 4, left. colored  arrows). Subparallel layers/formations are combined in one series and are interpolated using one scalar field/potential field where a certain isovalue of this field corresponds to the location of a stratigraphic boundary (Fig. 4, right). The measured orientations represent the gradient of the field (Fig. 4, right). Layers that are separated by an unconformity are modeled in separate series. The same accounts for faults where each fault is modeled as its own series. The order of the series and the formations from old at the bottom and young at the top with faults above the series and formations define the stratigraphic pile. This will be introduced in more detail during the modeling of basic structures.

The interpolation in GemPy is meshless working with a regular grid where the resulting layer or stratigraphic unit is assigned to the respective grid cell (voxel). The actual resulting structural geological model is therefore a so-called `lith_block` represented by a NumPy array. A marching-cube algorithm is applied to calculate the 2.5D surface meshes that can be visualized in PyVista [(Sullivan and Kaszynski, 2019)](https://doi.org/10.21105/joss.01450) and that can be used for post-processing in GemGIS.


<img src="../data/images/fig4.png" />

Fig. 4: Structural geological model showing the input data points, input orientations (arrows) and resulting meshes (continuous colored lines) with the resulting `lith_block` (left) and scalar field (right). 


The GemGIS package has been developed as an Add-On to GemPy to accelerate and simplify the processing of spatial data as input data for the structural geological modeling with GemPy (Fig 5). It is wrapping and extends the functionality of already well-known spatial data open-source Python packages such as [GeoPandas](https://geopandas.org/en/stable/docs.html) [(Bosche et al., 2022)](https://doi.org/10.5281/zenodo.2585848) or [Rasterio](https://rasterio.readthedocs.io/en/latest/) (Gillies et al., 2019). In addition, it provides functions to utilize the created structural geological models further (creating virtual boreholes, creating depth maps, etc.). 

<img src="../data/images/fig5.png" />

Fig. 5: From Maps to Models. Source of left map unknown.


<a id='resources-for-gemgis-gempy'></a>

## Resources for GemPy and GemGIS

The following resources are recommended for GemPy and GemGIS. In addition, it is recommended to join the Software Underground (SWUNG) Slack Workspace. This is "the place for scientists and engineers that love rocks and computers". In this community with more than 4000 enthusiasts, there are dedicated channels for the structural geological modeling with #gempy, #geospatial data, #open-geosciences and many more. The annual "Transform" conference is the official conference of the Software Underground with many tutorial sessions introducing the different open-source packages that have emerged from this community among other presentations and lightning talks. 

1. GemPy
    1. [Github Repository](https://github.com/cgre-aachen/gempy)
    2. [GemPy Documentation](https://www.gempy.org/)
    3. [GemPy Installation](https://www.gempy.org/installation)
    4. [GemPy Publication](https://gmd.copernicus.org/articles/12/1/2019/)
    5. [Tutorial: Geological Modeling with GemPy @Transform 2020](https://www.youtube.com/watch?v=n0btC5Zilyc&t=1s)
    6. [Tutorial: Geological Modeling with GemPy @Transform 2020](https://www.youtube.com/watch?v=1oS6xTJkRwo)
    7. [Tutorial: Geological Modeling with GemPy ](https://www.youtube.com/watch?v=7P6WrBOaHSM)
    
2. GemGIS
    1. [Github Repository](https://github.com/cgre-aachen/gemgis)
    2. [GemGIS Documentation](https://gemgis.readthedocs.io/)
    3. [GemGIS Installation](https://gemgis.readthedocs.io/en/latest/getting_started/installation.html)
    4. [GemGIS Publication](https://joss.theoj.org/papers/10.21105/joss.03709)
    
3. Software Underground Community
    1. [Software Underground Website](https://softwareunderground.org/)
    2. [Software Underground Slack](https://softwareunderground.org/slack)
    3. [Transform Conference](https://transform.softwareunderground.org/overview)
    

<div class="alert alert-block alert-warning">
<b>Important:</b> Please always check the latest installation instructions for GemPy and GemGIS referenced above. As both libraries depend on packages like NumPy, Pandas, GeoPandas, Rasterio, PyVista, and others, breaking changes may occur that have not been fixed in the latest release yet. In that case we would appreciate a heads up by opening a new issue on Github (https://github.com/cgre-aachen/gempy/issues or https://github.com/cgre-aachen/gemgis/issues). As of early 2023, the GemGIS installation was stable for versions 1.0.X, for GemPy for version 2.3.X. GemPy versions 2.2.X experienced several import errors due to breaking changes introduced by several dependencies. 
    
</div>


<a id='installation-of-gempy-and-gemgis'></a>

## Installation of GemPy and GemGIS

For extensive installation instructions, see [GemPy Installation](https://www.gempy.org/installation) and [GemGIS Installation](https://gemgis.readthedocs.io/en/latest/getting_started/installation.html). 

Both packages are available on PyPi and conda:
1. GemPy
    1. [GemPy PyPi](https://pypi.org/project/gempy/)
    2. [GemPy conda](https://anaconda.org/conda-forge/gempy)
2. GemGIS
    1. [GemGIS PyPi](https://pypi.org/project/gemgis/)
    2. [GemGIS conda](https://anaconda.org/conda-forge/gemgis)

In this section, we will provide a short version of the installation instructions to get you started. 

### Getting Anaconda
It is recommended to use the [latest Anaconda distribution](https://www.anaconda.com/products/distribution). 

### Creating Virtual Environment

It is recommended to create a new virtual environment when using GemGIS to avoid conflicts with already existing projects. This step and all following steps will be performed within the Anaconda Prompt (Anaconda3). 

Creating a new environment in Anaconda with fixed Python version:

```python
conda create -n gemgis python==3.10
```

### Activate Virtual Environment

Activate the virtual environment:

```python
conda activate gemgis
```

The gemgis environment now replaced the base environment which is indicated by `gemgis` in front of your path.

### Installing Packages and Dependencies

<div class="alert alert-block alert-warning">
<b>Installation via pip:</b> An installation of GemGIS and all its dependencies via pip is currently not supported! However, GemGIS can be easily installed via `pip install gemgis` once all dependencies have successfully been installed.  
    
</div>

Several packages need to be installed in order to use GemGIS. It is recommended to use `Jupyter Notebooks` when working with GemGIS. Packages are installed using the `conda-forge` channel.

Install Jupyter Notebooks:

```python
conda install -c conda-forge jupyter
```

You can start a new kernel by executing `jupyter notebook` in your Anaconda Prompt. A new kernel will then open in your browser.

### Installing GemGIS via conda-forge
GemGIS and all its dependencies can be installed via conda-forge:

```python
conda install -c conda-forge gemgis
```

### Installing GemGIS and ist dependencies manually

Two of the main packages that GemGIS is dependent on are rasterio and GeoPandas. It is recommended to install these packages separately as they both depend on the GDAL translator library for raster and vector geospatial data. In addition, many smaller libraries like shaply or fiona will also be installed properly.

Install the latest versions of GeoPandas and Rasterio (as of 2023-01-01). Please mind the quotation marks that are necessary when specifying the version numbers:

```python
conda install -c conda-forge geopandas">=0.12.2" rasterio">=1.3.4" pyvista">=0.38.2"
```

```python
pip install gemgis
```

### Installing GemPy via pip

Once GemGIS has successfully been installed, GemPy can be installed via pip. Please mind that GemPy is an optional dependency of GemGIS.

```python
pip install gempy
```

If you want to use GemPy and/or GemGIS, do not forget to make the correct contributions. 

For GemPy, please cite the publication by [de la Varga et al. (2019)](https://doi.org/10.5194/gmd-12-1-2019):

```python
@article{delaVarga2019,
author = {de la Varga, M. and Schaaf, A. and Wellmann, F.},
title = {GemPy 1.0: open-source stochastic geological modeling and inversion},
journal = {Geoscientific Model Development},
volume = {12},
year = {2019},
number = {1},
pages = {1-32},
url = {https://gmd.copernicus.org/articles/12/1/2019/},
doi = {https://doi.org/10.5194/gmd-12-1-2019}
}
```

For GemGIS, please cite the publication by [Jüstel et al. (2022)](https://doi.org/10.21105/joss.03709):

```python
@article{Jüstel2022,
 doi = {https://doi.org/10.21105/joss.03709},
 url = {https://doi.org/10.21105/joss.03709},
 year = {2022}, 
         publisher = {The Open Journal},
 volume = {7},
 number = {73},
 pages = {3709},
 author = {Jüstel, A. and Endlein Correira, A. and Pischke, P. and de la Varga, M. and Wellmann, F.},
 title = {GemGIS - Spatial Data Processing for Geomodeling},
 journal = {Journal of Open Source Software}
 }
```

# Next Notebooks

In the next four notebooks, the principles of GemPy will be explained. Here, you will model planar layers, folded layers, faulted layers and unconformable layers truncated by an unconformity. No topography is considered in these models.

[Take me to the next notebooks]

<img src="../jose/images/fig1.png" />

# References

Bossche, M. F.; McB., James; Wasserman, J.; Richards, M; et al. (2022): geopandas/geopandas: v0.12.2: Zenodo. [https://doi.org/10.5281/zenodo.2585848](https://doi.org/10.5281/zenodo.2585848)

de La Varga, Miguel ; Schaaf, Alexander; Wellmann, Florian (2019): GemPy 1.0: open-source stochastic geological modeling and inversion. 0001_delaVarga2019_GemPy.pdf. In: Geosci. Model Dev. 12 (1), S. 1–32. [https://doi.org/10.5194/gmd-12-1-2019](https://doi.org/10.5194/gmd-12-1-2019).

Gillies, S.; et al. (2019): Rasterio: geospatial raster I/O for Python programmers. [https://github.com/rasterio/rasterio](https://github.com/rasterio/rasterio).

Jüstel, A.; Endlein Correira, A.; Pischke, M.; de La Varga, M.; Wellmann, F. (2022): GemGIS - Spatial Data Processing for Geomodeling. In: JOSS 7 (73), S. 3709. [https://doi.org/10.21105/joss.03709](https://doi.org/10.21105/joss.03709).

Lajaunie, Christian; Courrioux, Gabriel; Manuel, Laurent (1997): Foliation fields and 3D cartography in geology: Principles of a method based on potential interpolation. In: Math Geol 29 (4), S. 571–584. [https://doi.org/10.1007/BF02775087](https://doi.org/10.1007/BF02775087).

Steno, N. (1669): De solido intra solidium naturaliter contento dissertationis prodromus. Florence.

Sullivan, C.; Kaszynski, Alexander (2019): PyVista: 3D plotting and mesh analysis through a streamlined interface for the Visualization Toolkit (VTK). In: JOSS 4 (37), S. 1450. DOI: [https://doi.org/10.21105/joss.01450](https://doi.org/10.21105/joss.01450).

Wellmann, Florian; Caumon, Guillaume (2018): 3-D Structural geological models: Concepts, methods, and uncertainties. In: Cedric Schmelzbach (Hg.): Advances in Geophysics, Bd. 59: Elsevier (Advances in Geophysics), S. 1–121. [https://doi.org/10.1016/bs.agph.2018.09.001](https://doi.org/10.1016/bs.agph.2018.09.001).